In [1]:
import pandas as pd # Beautiful tool for data wrangling! e.g. '!pip install pandas' from a Notebook
# See https://mariadb.com/blog/how-connect-python-programs-mariadb e.g. '!pip install mysql' from Notebook
import MySQLdb 
import re
from collections import Counter
import os
from collections import OrderedDict
import pickle

pd.set_option("display.max_rows",35) # Useful when having large Pandas DataFrames like we do here

In [2]:
conn = MySQLdb.connect(user='mos', passwd='', db='monuments_db', charset='utf8', use_unicode=True)
cursor = conn.cursor()
cursor.execute("SET NAMES utf8")

0

In [5]:
def create_mapping_table_from_source_tables(table):
    """Takes a Pandas DataFrame object and writes wikitables to files named after the table they are
    produced from to {the current Directory}/wikitables/
    """
    sql = 'SELECT * FROM `{}`'.format(table)
    df = pd.io.sql.read_sql(conn.escape_string(sql), conn)
    
    country_fields = list(df.keys())
    country_values = list(df.ix[0]) # get first row in table as example data from 
    country_data = list(zip(country_fields, country_values))
    
    table_header = '{| class="wikitable" style="width: 675px;\n'
    table_name = '|+ '+table + "\n"
    # create table columns
    table_columns = "! heritage field\n! example value\n! Wikidata property\n! Conversion\n! Comment\n|-\n"
    table_rows = []
    for (field, value) in zip(country_fields, country_values):
        row="| "+ str(field) + "\n| " + str(value) + "\n|\n| \n| \n|-\n"
        table_rows.append(row)
    
    table_rows_str = "".join(table_rows)
    # Fill in examples values from the first record in the table
    table_footer = "\n|}"
    wikitable = table_header + table_name + table_columns + table_rows_str[:-1] + table_footer
    
    if os.path.isdir("./mappingtables"):
        out = open("./mappingtables/" + table + ".mappingtable","w")
        out.write(wikitable)
        out.flush()
        
        print("Directory ./files exists. Wrote file {}".format(out.name))
        out.close()
    else:
        os.mkdir("./mappingtables")
        with open("./mappingtables/" + table + ".mappingtable","w") as out:
            out.write(wikitable)
            print("./mappingtables doesn't exist. Created it and wrote file {}".format(out.name))

In [6]:
final_tables = pickle.load(open("./final_tables.pickle","rb"))
for table in final_tables:
    create_mapping_table_from_source_tables(table)

./mappingtables doesn't exist. Created it and wrote file ./mappingtables/monuments_hk-hb_(en).mappingtable
Directory ./files exists. Wrote file ./mappingtables/monuments_nl-prov_(nl).mappingtable
Directory ./files exists. Wrote file ./mappingtables/monuments_at_(de).mappingtable
Directory ./files exists. Wrote file ./mappingtables/monuments_ee_(et).mappingtable
Directory ./files exists. Wrote file ./mappingtables/se_fornmin.mappingtable
Directory ./files exists. Wrote file ./mappingtables/se_bbr.mappingtable
Directory ./files exists. Wrote file ./mappingtables/monuments_it_(it).mappingtable
Directory ./files exists. Wrote file ./mappingtables/monuments_be-wal_(en).mappingtable
Directory ./files exists. Wrote file ./mappingtables/monuments_th_(th).mappingtable
Directory ./files exists. Wrote file ./mappingtables/monuments_sk_(de).mappingtable
Directory ./files exists. Wrote file ./mappingtables/monuments_jo_(ar).mappingtable
Directory ./files exists. Wrote file ./mappingtables/monuments

KeyboardInterrupt: 